# Manipulating Tick Data with pandas

We will work with data from [QuantQuote](https://quantquote.com/historical-stock-data).

- Dimensions: date, time, stock symbol
- Metrics: opening, high, low and closing prices, as well as trade volume
- Frequency: daily
- Dates: 1998 to 2015
- Scope: 500 stock symbols that constitute the S&P500 as of Dec 2015.

Let's get the data

In [ ]:
from urllib.request import urlretrieve
from zipfile import ZipFile
import os.path

def download(url):
    local_fname = url.split('/')[-1]
    if os.path.exists(local_fname):
        return local_fname
    else:
        urlretrieve(url, local_fname)
        return local_fname

data_url = 'http://quantquote.com/files/quantquote_daily_sp500_83986.zip'
metadata_url = 'https://quantquote.com/docs/QuantQuote_Minute.pdf'

# Download data
data_fname = download(data_url)
# Extract the data
with ZipFile(data_fname) as zf:
    zf.extractall()

# Download PDF with the metadata
metadata_fname = download(metadata_url)

For each one of the 500 stock symbols, we have a file. Here is a sample:

In [ ]:
from pprint import pprint

data_dir = os.path.join('quantquote_daily_sp500_83986', 'daily')
pprint(os.listdir(data_dir)[:10])

From the downloaded PDF (see `metadata_fname`) we can obtain the field names (and their descriptions):

In [ ]:
fieldnames = [
    'date',
    'time',
    'open',
    'high',
    'low',
    'close',
    'volume' 
]

## Building the dataset for a single stock symbol

We can now easily import the data of a single stock, for example, Apple (AAPL). We will drop the `time` field because it's useless.

First, let's get set to work with dataframes and matplotlib visualizations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
print(plt.style.available)

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
def import_data(symbol, fieldnames=fieldnames, set_index=True, add_symbol=False):
    data_path = os.path.join('quantquote_daily_sp500_83986',
                             'daily',
                             'table_' + symbol + '.csv')
    data = pd.read_csv(data_path,
                       names=fieldnames).drop('time', axis=1)
    data['date'] = pd.to_datetime(data.date, format='%Y%m%d')
    if set_index:
        data.set_index('date', inplace=True)
    if add_symbol:
        data['symbol'] = symbol
    return data

aapl = import_data('aapl')

In [ ]:
print(aapl.info())
print(aapl.describe())

We can quickly look at things like the closing prices:

In [ ]:
aapl.close.plot(title='AAPL closing prices')

This kind of plot where you have a lot of data points in the x-axis begs for interactivity: sometimes we want to inspect prices dates where something happened. Bokeh can help greatly here, allowing you to zoom in the dates you are interested in easily.

In [ ]:
import bokeh.charts, bokeh.io
bokeh.io.output_notebook()

In [ ]:
p = bokeh.charts.Line(aapl.close.reset_index(), x='date', y='close')
p.notebook(True).show()

We can also look at the relative difference between open and close prices:

In [ ]:
_df = (aapl.close - aapl.open) / aapl.open
print(_df.describe())
_df.plot(title='AAPL relative difference between close and open prices')

In [ ]:
_df.hist(bins=50)

## Building a dataset for all stocks

First, you should check if the data is "too big" just by looking at its size. It's 35M compressed, so we will be fine loading it all in memory.

In [ ]:
_data_dir = os.path.join('quantquote_daily_sp500_83986', 'daily')

def make_dataset(data_dir=_data_dir):      
    data_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    symbols = [os.path.splitext(f)[0].split('_')[1] for f in data_files]
    df_by_symbol = (import_data(s, set_index=False, add_symbol=True)
                    for s in symbols)
    df = pd.concat(df_by_symbol)
    
    # Encode categorical variables efficiently
    df['symbol'] = df.symbol.astype('category')
    # Set an index and assert it is well behaved
    df = df.set_index(['symbol', 'date']).sort_index()
    assert df.index.is_unique and df.index.is_monotonic
    return df

In [ ]:
%time df = make_dataset()

We can now access the data in various ways:

In [ ]:
df.head()

In [ ]:
df.loc['aapl'].head()

Let's say now we want to visualize the closing prices for Microsoft (MSFT) and Apple (AAPL) stock:

In [ ]:
_symbols = ['aapl', 'msft']
_df = df.loc[_symbols].close.unstack('symbol')

print(_df.head())
_df.plot()

The main thing to remember about plotting with pandas is __the x-axis corresponds to the index and each column corresponds to a different series__.

Log-scale on the closing price might help clear things up here:

In [ ]:
import numpy as np

# one way of using log scale in the y-axis
np.log(_df).plot()

The plots handle missing data gracefully too:

In [ ]:
_symbols = ['aapl', 'msft', 'goog', 'yhoo']
_df = df.loc[_symbols].close.unstack('symbol')

print(_df.head())
_df.plot(logy=True) # another way of using log scale in the y-axis

Notice how the legend covers part of the plot. Pandas plots are just matplotlib plots and thus you can [modify their legend](http://matplotlib.org/users/legend_guide.html) the same way:

In [ ]:
# Sets the upper-left corner of the legend box at position bbox_to_anchor
# bbox_to_anchor=(0.5,0.5) would be the center of the plot

_df.plot(logy=True).legend(bbox_to_anchor=(1.1, 0.5), loc='upper left')

# exercise: restrict the plot to the period after the first iPhone release (June 2007)

We can also plot one stock against another:

In [ ]:
from pandas.tools.plotting import scatter_matrix

scatter_matrix(_df, alpha=0.2, figsize=(9, 9), diagonal='kde')

You may also want to know how much you would have made over time from 1 dollar invested on each stock on Jan 1st 2006:

In [ ]:
_symbols = ['aapl', 'msft', 'goog', 'yhoo']
_df = df.loc[_symbols].close.unstack('symbol').loc['2006':]


_df.div(_df.iloc[0]).plot(kind='area')

In [ ]:
# Only on pandas 0.17+

_df.div(_df.iloc[0]).plot.area()

## The most traded stocks

We could want to restrict attention to the most traded stocks. To find 20 most traded stocks:

In [ ]:
most_traded_symb = (
    df.groupby(level='symbol')
      .sum()
      .sort_values(by='volume', ascending=False)
      .index
      .tolist()
)

most_traded_symb[:20]

Why isn't Google there? Let's see where Google is in the list:

In [ ]:
most_traded_symb.index('goog')

Wow, not even close to top 20 most traded. I wonder why.

In [ ]:
# exercise: select different combinations of stock symbols and dates
# See http://pandas.pydata.org/pandas-docs/stable/advanced.html#using-slicers

top20 = df.loc[(most_traded_symb[:20],slice(None)), ['open', 'close']]

top20.head()

In [ ]:
top20.close.unstack('symbol').head()

We may want to exclude AAPL from a visualizations of returns over time on a dollar invested:

In [ ]:
_df0 = top20.drop('aapl').unstack(level='symbol')
_df0.div(_df0.iloc[0]).close.plot.area().legend(bbox_to_anchor=(1.1, 0.5), loc='upper left', ncol=3)

In [ ]:
# Exercise: which days of the week see the most trade? What about days of the month?

## What the Olympics have to do with making bets

We will now look into a visual way of choosing among multiple alternatives that give you a random return.

Before that, a short detour into athletics: in the 2012 summer olympics, these were the number of gold, silver and bronze medals for the US, China, Russia and Great Britain, respectively:

- US: 46,29,29
- China: 38,27,22
- Russia: 24,25,33
- Great Britain: 29, 17,19

How do these olympic teams compare? Who is better: Great Britain or Russia? Here is a visual answer:

In [ ]:
medals = {'US': [46,29,29],
          'China': [38,27,22],
          'Russia': [24,25,33],
          'Great Britain': [29, 17,19],
          'medals': ['gold', 'silver', 'bronze']}

summer_olymp = pd.DataFrame(medals).set_index('medals')
summer_olymp

In [ ]:
# Note that to interpret the index/x-axis, you need to know we are doing a cumulative sum
# It's easy to do it this way, but it can confuse your audience. Be careful.

summer_olymp.cumsum().plot.bar()

It follows from above that, unanmbiguously, US > China > {Russia, Great Britan}. However, whether Russia beats Great Britain or vice-versa depends on the relative weights of each type of medal.

Note that Great Britan has more gold medals than Russia, but Russia has more medals than Great Britan when we count gold and silver, or when we count all three types of medals.

The following plot is another way of visualizing the same information.

In [ ]:
summer_olymp.transpose().plot.bar(stacked=True).legend(bbox_to_anchor=(1.1, 0.5), loc='upper left', ncol=1)

We can use similar ideas when making bets, choosing models, etc. as long as the returns from our choice can be quantified in a probability distribution.

For simplicity, imagine stock returns are independent, and that you need to choose one stock to "bet on". Betting on a stock means that you will buy 1 dollar of it when the market opens, and sell what you could buy with that 1 dollar at the end of the day. Again, for simplicity, restrict attention to the stocks ['ge', 'nflx', 'yhoo'].

One way to proceed could be as follows. We compare the empirical cumulative distribution function (ECDF) of each alternative: the alternative with the "lowest" ECDF (if there is one), is the best alternative. That is because that "lowest" ECDF unambiguously puts more mass on higher-value outcomes. In particular, if we ranked the alternatives by the mean, median, quartiles, etc., all these measures would agree that the alternative with the lowest ECDF would be the best. In that case, we say this lowest ECDF stochastically dominates all other distributions in a first-order sense.

If there is no uniformly lowest ECDF, then we need to analyze the graph and see where various ECDFs cross each other to make a judgement call.

You may want to look up the definition of [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) and [first-order stochastic dominance](https://en.wikipedia.org/wiki/Stochastic_dominance#First-order_stochastic_dominance).

In [ ]:
spread = ((df.close - df.open) / df.open).loc[['ge', 'nflx', 'yhoo']]

In [ ]:
def plot_cdfs(spread):
    binned = (np.round(spread * 100) /100).reset_index().groupby(['symbol', 0]).count()
    binned.index.names = ['symbol', 'bin']
    numerator = binned.unstack(level='symbol').cumsum()
    denominator = binned.unstack(level='symbol').sum()
    return numerator.div(denominator).plot()

plot_cdfs(spread)

As you can see, there is no clear winner: the options with good returns are also the options with high costs. But what if you could somehow guess if the stock will go up or down in that day. In that case:

In [ ]:
plot_cdfs(np.abs(spread))

And in this case Netflix looks like the winner.

## Panel data

In [ ]:
pan = df.to_panel()
pan

In [ ]:
# Note that dates go in the columns, for some reason

pan.loc['close'].head()

In [ ]:
# This fixes it

pan.loc['close'].transpose().head()

Another way to build a panel:

In [ ]:
dd = {sym: df.loc[sym] for sym in most_traded_symb[:20]}
panel = pd.Panel(dd)

In [ ]:
panel.loc['yhoo'].head()

# Note that this time the dates came correctly in the rows

In [ ]:
panel.apply(lambda x: x.mean(), axis='major_axis')

In [ ]:
df.loc['nflx'].head()